In [ ]:
'''
Для выбранного датасета с сайта https://github.com/YBI-Foundation/Dataset 

1. Напишите одну функцию и примените ее к сгруппированным данным с помощью .apply()
2. Напишите другую функцию и примените ее к сгруппированным данным с помощью .transform()
3. Напишите третью функцию и примените ее ко всем данным с помощью .map()
4. Сгруппируйте данные по дате (если применимо) и используйте resample для агрегирования в более крупный период
5. Если в вашем датасете нет дат, используйте выборку из БД dvdrental для выполнения этого пункта

Срок до 1 октября 2024 г
'''

In [1]:
import pandas as pd
import numpy as np

In [232]:
df_Bike_Prices = pd.read_csv(r"https://raw.githubusercontent.com/YBI-Foundation/Dataset/refs/heads/main/Bike%20Prices.csv")

In [233]:
df_Bike_Prices = df_Bike_Prices.drop_duplicates()
df_Bike_Prices

,Brand,Model,Selling_Price,Year,Seller_Type,Owner,KM_Driven,Ex_Showroom_Price
0,TVS,TVS XL 100,30000,2017,Individual,1st owner,8000,30490.0
1,Bajaj,Bajaj ct 100,18000,2017,Individual,1st owner,35000,32000.0
2,Yo,Yo Style,20000,2011,Individual,1st owner,10000,37675.0
3,Bajaj,Bajaj Discover 100,25000,2010,Individual,1st owner,43000,42859.0
4,Bajaj,Bajaj Discover 100,24999,2012,Individual,2nd owner,35000,42859.0
...,...,...,...,...,...,...,...,...
1056,Royal,Royal Enfield Electra 5 S,90000,2012,Individual,1st owner,40000,NaN
1057,Hero,Hero Honda Hunk,20000,2010,Individual,1st owner,17000,NaN
1058,Bajaj,Bajaj Pulsar 220 DTS-i,60000,2014,Individual,1st owner,16000,NaN
1059,Hero,Hero Honda CBZ extreme,40000,2009,Individual,1st owner,50000,NaN


In [234]:
# Добавим (сгенерируем) даты продажи для п. 4.
# т.к. у нас в прайсе 1055 позиций, используем опцию periods=1055
df_Bike_Prices['Date_of_sale'] = pd.date_range(start='2020-01-01', periods=1055, freq='D')

In [235]:
# Немного изменим порядок столбцов
df_Bike_Prices = df_Bike_Prices[['Date_of_sale', 'Brand', 'Model', 'Year', 'KM_Driven', 'Selling_Price']]

In [236]:
df_Bike_Prices.head()

,Date_of_sale,Brand,Model,Year,KM_Driven,Selling_Price
0,2020-01-01,TVS,TVS XL 100,2017,8000,30000
1,2020-01-02,Bajaj,Bajaj ct 100,2017,35000,18000
2,2020-01-03,Yo,Yo Style,2011,10000,20000
3,2020-01-04,Bajaj,Bajaj Discover 100,2010,43000,25000
4,2020-01-05,Bajaj,Bajaj Discover 100,2012,35000,24999


In [237]:
# Напишем лямбда-функции скидок: 
discount_5 = lambda x: round(x*0.05, 2) 
discount_10 = lambda x: round(x*0.1, 2) 
discount_15 = lambda x: round(x*0.15, 2) 

In [239]:
# 1. Применим лямбда-функцию 5% скидки методом .apply() 
(
    df_Bike_Prices[(df_Bike_Prices['Brand'] == 'Honda')] # Включим фильтрацию по Хонде 
    .groupby(['Brand', 'Model'])['Selling_Price'] # Группируем по бренду и модели 
    .mean() # Считаем среднюю цену модели 
    .apply(discount_5) # Считаем средний размер скидки для модели 
)

Brand  Model                       
Honda  Honda Activa 125                1875.00
       Honda Activa 125 [2016-2017]    2337.50
       Honda Activa 3G                 2023.33
       Honda Activa 4G                 2300.00
       Honda Activa 5G                 2783.33
       Honda Activa [2000-2015]        1245.66
       Honda Activa i                  1550.00
       Honda Activa i [2016-2017]      1750.00
       Honda Aviator                   1592.86
       Honda CB Hornet 160R            3434.01
       Honda CB Shine                  1050.00
       Honda CB Shine SP               2444.44
       Honda CB Trigger                2016.67
       Honda CB Twister                1650.00
       Honda CB Unicorn                2075.00
       Honda CB Unicorn 150            2507.14
       Honda CB Unicorn 160            3062.50
       Honda CB Unicorn Dazzler        1616.67
       Honda CB twister                1025.00
       Honda CBF Stunner               1800.00
       Honda CBR 150    

In [240]:
# 2. Применим лямбда-функцию 10% скидки методом .transform() 
(
    df_Bike_Prices[(df_Bike_Prices['Brand'] == 'Yamaha')] # Включим фильтрацию по Ямахе 
    .groupby(['Brand', 'Model'])['Selling_Price'] # Группируем по бренду и модели 
    .mean() # Считаем среднюю цену модели 
    .transform(discount_10) # Считаем средний размер скидки для модели 
)

Brand   Model                            
Yamaha  Yamaha Alpha                          2500.00
        Yamaha Cygnus Ray ZR                  5600.00
        Yamaha FZ  v 2.0                      6000.00
        Yamaha FZ 16                          7500.00
        Yamaha FZ S                           4000.00
        Yamaha FZ S V 2.0                     6556.25
        Yamaha FZ S [2012-2016]               4428.57
        Yamaha FZ V 2.0                       5375.00
        Yamaha FZ16                           3541.67
        Yamaha FZ25                          12357.14
        Yamaha Fascino 110                    5175.00
        Yamaha Fazer                          5000.00
        Yamaha Fazer 25                      12666.67
        Yamaha Fazer Dlx                      5800.00
        Yamaha Fazer FI V 2.0 [2016-2018]     6500.00
        Yamaha Fazer [2009-2016]              3928.57
        Yamaha Gladiator                      1500.00
        Yamaha RX135                    

In [241]:
# 3. Применим лямбда-функцию 15% скидки методом .map() ко всем данным 

df_Bike_Prices['Discount_15%'] = df_Bike_Prices['Selling_Price'].map(discount_15)

/tmp/ipykernel_6788/950482349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Bike_Prices['Discount_15%'] = df_Bike_Prices['Selling_Price'].map(discount_15)


In [ ]:
# Хочу добавить в датафрейм столбец со скидкой, выдаётся предупреждение, хотя делаю аналогично как было в лекции:
# df_all['diff_disc'] = df_all['shop_name'].map(shops_dict) 
# Так и не смог с этим разобраться. 

In [242]:
# Пробую дабавить ещё один столбец с итоговой ценой, такое же предупреждение. 

df_Bike_Prices['Final_price'] = df_Bike_Prices['Selling_Price'] - df_Bike_Prices['Discount_15%']

/tmp/ipykernel_6788/385706277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Bike_Prices['Final_price'] = df_Bike_Prices['Selling_Price'] - df_Bike_Prices['Discount_15%']


In [243]:
# Но датафрейм выводится без ошибок. 

df_Bike_Prices

,Date_of_sale,Brand,Model,Year,KM_Driven,Selling_Price,Discount_15%,Final_price
0,2020-01-01,TVS,TVS XL 100,2017,8000,30000,4500.00,25500.00
1,2020-01-02,Bajaj,Bajaj ct 100,2017,35000,18000,2700.00,15300.00
2,2020-01-03,Yo,Yo Style,2011,10000,20000,3000.00,17000.00
3,2020-01-04,Bajaj,Bajaj Discover 100,2010,43000,25000,3750.00,21250.00
4,2020-01-05,Bajaj,Bajaj Discover 100,2012,35000,24999,3749.85,21249.15
...,...,...,...,...,...,...,...,...
1056,2022-11-16,Royal,Royal Enfield Electra 5 S,2012,40000,90000,13500.00,76500.00
1057,2022-11-17,Hero,Hero Honda Hunk,2010,17000,20000,3000.00,17000.00
1058,2022-11-18,Bajaj,Bajaj Pulsar 220 DTS-i,2014,16000,60000,9000.00,51000.00
1059,2022-11-19,Hero,Hero Honda CBZ extreme,2009,50000,40000,6000.00,34000.00


In [244]:
# 4. Группируем данные по дате продаж. Агрегируем цену продаж (считаем сумму) 
sales = df_Bike_Prices.groupby('Date_of_sale').agg({'Selling_Price':'sum'})
sales

,Selling_Price
Date_of_sale,
2020-01-01,30000
2020-01-02,18000
2020-01-03,20000
2020-01-04,25000
2020-01-05,24999
...,...
2022-11-16,90000
2022-11-17,20000
2022-11-18,60000


In [245]:
# Сумма продаж по месяцам 
sales.resample('M').sum()

,Selling_Price
Date_of_sale,
2020-01-31,762999
2020-02-29,888989
2020-03-31,965600
2020-04-30,1076500
2020-05-31,1233999
2020-06-30,1068000
2020-07-31,1022500
2020-08-31,1377000
2020-09-30,1120000


In [246]:
# Сумма продаж по годам 
sales.resample('Y').sum()

,Selling_Price
Date_of_sale,
2020-12-31,13540387
2021-12-31,31197685
2022-12-31,18260007
